In [ ]:
!pip install kfp
!pip install google-cloud-pipeline-components
!pip install gcsfs
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.2 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.7.0-py3-none-any.whl size=610419 sha256=979e13403e475a75cd0d2c6c0807f7e88080860e3663fbd46c7d9dfc06184ba8
  Stored in directory: /root/.cache/pip/wheels/9e/7d/a4/f9d013e82681c9746ef10de3b00456163577a99279c5ed673d
  Created wheel for kfp-server-api: filename=kfp_server_api-2.0.5-py3-none-any.whl size=114733 sha256=853a9f0aa65056c353ac70fab871dafdc403ff335fe89b40058f0aa93eb606db
  Stored in 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [29]:
from kfp.v2.dsl import pipeline, component, InputPath, OutputPath, Dataset, Output, Model, Metrics
from kfp.v2 import compiler
from google.cloud import aiplatform

# Set parameters
project_id = 'ise543-final-project-421906'
location = 'us-central1'
aiplatform.init(project=project_id, location=location)



In [30]:
from kfp.v2.dsl import Artifact
from kfp.v2.dsl import Input
from kfp.v2.dsl import Model

@component(packages_to_install=["pandas", "numpy", "fsspec", "gcsfs"])
def impute_multiple_features_training(training_dataset_path: str,
                                      imputed_dataset_path: OutputPath('Dataset'),
                                      feature_medians: Output[Artifact]):
    import pandas as pd

    df = pd.read_csv(training_dataset_path)
    features_to_impute = ['cigsPerDay', 'totChol', 'glucose', 'BMI', 'heartRate']
    medians = {}
    for feature in features_to_impute:
        median_value = df[feature].median()
        df[feature].fillna(median_value, inplace=True)
        medians[feature] = median_value

    categorical_features_to_impute = ['education', 'BPMeds']
    modes = {}
    for feature in categorical_features_to_impute:
        mode_value = df[feature].mode()[0]
        df[feature].fillna(mode_value, inplace=True)
        modes[feature] = mode_value

    feature_medians.metadata['medians'] = medians
    feature_medians.metadata['modes'] = modes

    df.to_csv(imputed_dataset_path, index=False)


/usr/local/lib/python3.10/dist-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


In [31]:
@component(packages_to_install=["pandas", "numpy", "fsspec", "gcsfs"])
def impute_multiple_features_validation(validation_dataset_path: str,
                                        imputed_dataset_path: OutputPath('Dataset'),
                                        feature_medians: Input[Artifact]):
    import pandas as pd

    df = pd.read_csv(validation_dataset_path)
    medians = feature_medians.metadata['medians']
    modes = feature_medians.metadata['modes']

    for feature, median_value in medians.items():
        df[feature].fillna(median_value, inplace=True)

    for feature, mode_value in modes.items():
        df[feature].fillna(mode_value, inplace=True)

    df.to_csv(imputed_dataset_path, index=False)

In [32]:
@component(packages_to_install=["pandas", "numpy", "fsspec", "gcsfs"])
def perform_initial_data_preparation(input_dataset_path: InputPath('Dataset'),
                                     output_dataset_path: OutputPath(Dataset)):
    import pandas as pd
    import numpy as np

    df = pd.read_csv(input_dataset_path)
    df = df.drop(columns=["a1c"])
    df["totChol"] = df["totChol"].clip(upper=700)
    df["BMI"] = df["BMI"].clip(upper=50)
    df["totChol"] = np.log(df["totChol"]+1)
    df["income"] = np.log(df["income"]+1)
    df.loc[df['currentSmoker'] == 0, 'cigsPerDay'] = 0
    df = pd.get_dummies(df, drop_first=True)
    df.to_csv(output_dataset_path, index=False)

In [33]:
@component(packages_to_install=["pandas", "numpy", "scikit-learn", "imbalanced-learn==0.11.0"])
def perform_SMOTE(input_df_path:  InputPath('Dataset'),
                  output_df_path: OutputPath('Dataset')):
    import pandas as pd
    import numpy as np
    from imblearn.over_sampling import SMOTE

    # Load the input dataset
    df = pd.read_csv(input_df_path)

    X = df.drop('TenYearCHD', axis = 1)
    y = df['TenYearCHD']

    # Perform SMOTE oversampling
    smote = SMOTE()
    X_smote, y_smote = smote.fit_resample(X, y)

    # Convert the oversampled feature set and target vector back into a DataFrame
    X_smote_df = pd.DataFrame(X_smote, columns=X.columns)
    y_smote_df = pd.DataFrame(y_smote, columns=['TenYearCHD'])

    # Re-join the features and the target into a single DataFrame
    oversampled_df = pd.concat([X_smote_df, y_smote_df], axis=1)

    # Save the re-joined, oversampled dataset to the specified OutputPath
    oversampled_df.to_csv(output_df_path, index=False)

In [34]:
@component(packages_to_install=["pandas", "scikit-learn", "joblib"])
def train_logistic_regression(training_dataset_path: InputPath('Dataset'),
                              trained_model_artifact: Output[Model]):
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    import joblib
    import os

    train_df = pd.read_csv(training_dataset_path)
    X_train = train_df.drop(['TenYearCHD', 'patientID'], axis=1)
    y_train = train_df['TenYearCHD']

    trained_model = LogisticRegression(max_iter=1000)
    trained_model.fit(X_train, y_train)

    os.makedirs(trained_model_artifact.path, exist_ok=True)
    joblib.dump(trained_model, os.path.join(trained_model_artifact.path, "model.joblib"))

In [35]:
@component(packages_to_install=["pandas", "scikit-learn", "joblib"])
def train_random_forest(training_dataset_path: InputPath('Dataset'),
                        trained_model_artifact: OutputPath(Model)):
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    import joblib
    import os

    train_df = pd.read_csv(training_dataset_path)
    X_train = train_df.drop(['TenYearCHD', 'patientID'], axis=1)
    y_train = train_df['TenYearCHD']

    trained_model = RandomForestClassifier(n_estimators=500, max_depth=4, random_state=42)
    trained_model.fit(X_train, y_train)

    model_path = os.path.join(trained_model_artifact, "model.joblib")
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    joblib.dump(trained_model, model_path)


In [36]:
@component(packages_to_install=["pandas", "scikit-learn", "joblib"])
def train_gradient_boosting(training_dataset_path: InputPath('Dataset'),
                            trained_model_artifact: Output[Model]):
    import pandas as pd
    from sklearn.ensemble import GradientBoostingClassifier
    import joblib
    import os


    train_df = pd.read_csv(training_dataset_path)
    X_train = train_df.drop(['TenYearCHD', 'patientID'], axis=1)
    y_train = train_df['TenYearCHD']


    trained_model = GradientBoostingClassifier(n_estimators=500, learning_rate=0.05, max_features = 5,
                                               max_depth=4, random_state=42)
    trained_model.fit(X_train, y_train)


    os.makedirs(trained_model_artifact.path, exist_ok=True)
    joblib.dump(trained_model, os.path.join(trained_model_artifact.path, "model.joblib"))


In [43]:
@component(packages_to_install=["pandas", "scikit-learn", "joblib", "numpy"])
def evaluate_model(test_dataset_path: InputPath('Dataset'),
                   model: Input[Model],
                   metrics: Output[Metrics]):
    import pandas as pd
    import joblib
    import numpy as np
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


    test_df = pd.read_csv(test_dataset_path)
    X_test = test_df.drop(['TenYearCHD', 'patientID'], axis=1)
    y_test = test_df['TenYearCHD']


    trained_model = joblib.load(model.path + "/model.joblib")

    y_pred = trained_model.predict(X_test)
    y_pred_2 = trained_model.predict_proba(X_test)[:, 1]
    auc_pr = roc_auc_score(y_test, y_pred_2)


    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')


    metrics.log_metric("accuracy", accuracy)
    metrics.log_metric("f1_score", f1)
    metrics.log_metric("AUC_PR", auc_pr)


/usr/local/lib/python3.10/dist-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


In [44]:
@pipeline(name='final-project-pipeline')
def final_project_pipeline(training_dataset_path: str, validation_dataset_path: str):

    # Perform imputation before data preparation
    training_imputation = impute_multiple_features_training(training_dataset_path=training_dataset_path)
    validation_imputation = impute_multiple_features_validation(validation_dataset_path=validation_dataset_path,
                                                                feature_medians=training_imputation.outputs['feature_medians'])

    # Data preparation using the output from the imputation step
    training_data_preparation = perform_initial_data_preparation(input_dataset_path=training_imputation.outputs['imputed_dataset_path'])
    validation_data_preparation = perform_initial_data_preparation(input_dataset_path=validation_imputation.outputs['imputed_dataset_path'])

    # Perform SMOTE oversampling on the training partition
    oversampled_training_data = perform_SMOTE(input_df_path=training_data_preparation.outputs['output_dataset_path'])

    # Training and evaluation using the prepared datasets
    trained_random_forest = train_random_forest(training_dataset_path=oversampled_training_data.outputs['output_df_path'])
    evaluate_model(test_dataset_path=validation_data_preparation.outputs['output_dataset_path'],
                   model=trained_random_forest.outputs['trained_model_artifact'])

    trained_logistic_regression = train_logistic_regression(training_dataset_path=oversampled_training_data.outputs['output_df_path'])
    evaluate_model(test_dataset_path=validation_data_preparation.outputs['output_dataset_path'],
                   model=trained_logistic_regression.outputs['trained_model_artifact'])

    trained_gradient_boosting = train_gradient_boosting(training_dataset_path=oversampled_training_data.outputs['output_df_path'])
    evaluate_model(test_dataset_path=validation_data_preparation.outputs['output_dataset_path'],
                   model=trained_gradient_boosting.outputs['trained_model_artifact'])


In [45]:
compiler.Compiler().compile(
    pipeline_func=final_project_pipeline,
    package_path='final_project_pipeline.json'
)

In [46]:
pipeline_job = aiplatform.PipelineJob(
    display_name='final_project_pipeline',
    template_path='final_project_pipeline.json',
    pipeline_root='gs://final-bucket-2',
    parameter_values={
        'training_dataset_path': 'gs://final-bucket-2/train_data.csv',
        'validation_dataset_path': 'gs://final-bucket-2/val_data.csv'
    },
    enable_caching=True
)

In [47]:
pipeline_job.run()


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/795244162107/locations/us-central1/pipelineJobs/final-project-pipeline-20240502235419
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/795244162107/locations/us-central1/pipelineJobs/final-project-pipeline-20240502235419')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/final-project-pipeline-20240502235419?project=795244162107
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/795244162107/locations/us-central1/pipelineJobs/final-project-pipeline-20240502235419 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/795244162107/locations/us